In [35]:
#ENUMERATION based on CSV file
#isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count
ISBN13 = 0
ISBN10 = 1
TITLE = 2
SUBTITLE = 3
AUTHOR = 4
CATEGORIES = 5
THUMBNAIL = 6
DESCRIPTION = 7
PUBLISHED_YEAR = 8
AVERAGE_RATING = 9
NUM_PAGES = 10
RATING_COUNT = 11



In [36]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)

# Define a function to generate embeddings
def get_embedding(data, precision="float32"):
   return model.encode(data, precision=precision).tolist()


<All keys matched successfully>


In [ ]:
import csv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

uri = "your link"
# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
database = client['book_recommendation']
collection = database['book']

def database_item(lines):
    #filing the dictionary
    
    line_data = {
        "ISBN13": lines[ISBN13],
        "ISBN10": lines[ISBN10],
        "TITLE": lines[TITLE],
        "SUBTITLE": lines[SUBTITLE],
        "AUTHOR": lines[AUTHOR],
        "CATEGORIES": lines[CATEGORIES],
        "THUMBNAIL": lines[THUMBNAIL],
        "DESCRIPTION": lines[DESCRIPTION],
        "PUBLISHED_YEAR": lines[PUBLISHED_YEAR],
        "AVERAGE_RATING": lines[AVERAGE_RATING],
        "NUM_PAGES": lines[NUM_PAGES],
        "RATING_COUNT": lines[RATING_COUNT],
        "EMBEDDING" : None
    }

    embed_string_list = [
        "TITLE : " + lines[TITLE],
        "SUBTITLE : " + lines[SUBTITLE],
        "AUTHOR : " + lines[AUTHOR],
        "CATEGORIES : " + lines[CATEGORIES],
        "DESCRIPTION : " + lines[DESCRIPTION],
        "PUBLISHED_YEAR : " + lines[PUBLISHED_YEAR],
        "AVERAGE_RATING : " + lines[AVERAGE_RATING]
    ]

    embed_string = " ".join(embed_string_list)

    embedding = get_embedding(embed_string)
    line_data["EMBEDDING"] = embedding
    return line_data

#uploading data to database

with open('data2.csv', mode='r') as file :
    reader = csv.reader(file)
    next(reader) #skip the first line
    book_array = []
    counter = 0
    ship = 0

    for lines in reader:
        line_data = database_item(lines)
        book_array.append(line_data)
        counter += 1

        if counter == 20:
            collection.insert_many(book_array)
            ship += 1
            print("The " + str(ship) + " shipping")
            book_array = []
            counter = 0
    
    collection.insert_many(book_array)
    ship += 1
    print("The " + str(ship) + " shipping")

        
    
    